In [1]:
!pip install tika

  Created wheel for tika: filename=tika-1.24-py3-none-any.whl size=32891 sha256=19602ca55178ae979445d8f4aa29d448595e8504a75d712bd64c38b2371e5c8f
  Stored in directory: /root/.cache/pip/wheels/ec/2b/38/58ff05467a742e32f67f5d0de048fa046e764e2fbb25ac93f3
Successfully built tika


In [2]:
file = r"/content/drive/MyDrive/sample_project/Sahih Al Bukhari Eng.pdf"
from tika import parser # importing parser modeule from tika library

file_data = parser.from_file(file)
text = file_data['content']
print(text)
  


2022-01-03 20:45:23,213 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2022-01-03 20:45:28,003 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2022-01-03 20:45:29,148 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...














































Sahih Al Bukhari Eng


SAHIH BUKHARI VOLUME 1 > BOOK 1: REVELATION 

Book 1: Revelation 

Volume 1, Book 1, Number 1: 

Narrated 'Umar bin Al-Khattab: 

I heard Allah's Apostle saying, "The reward of deeds depends upon the intentions and every person 

will get the reward according to what he has intended. So whoever emigrated for worldly benefits or 

for a woman to marry, his emigration was for what he emigrated for." 

Volume 1, Book 1, Number 2: 

Narrated 'Aisha: 

(the  mother  of  the  faithful  believers)  Al-Harith  bin  Hisham  asked  Allah's  Apostle  "O  Allah's 

Apostle! How is the Divine Inspiration revealed to you?" Allah's Apostle replied, "Sometimes it is (re-

vealed) like the ringing of a bell, this form of Inspiration is the hardest of all and then this state 

passes ' off after I have grasped what is inspired. Sometimes the Angel comes in the form of a man 

and talks to me and I grasp whatever he says." 'Aisha added: V

In [3]:
type(text)

str

### Text Preprocessing

In [13]:
# convert text to lowercase
text = text.lower()

In [14]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

In [15]:
# load the appostrophes connecting words
nego_file = open("/content/drive/MyDrive/sample_project/nego.txt","r")
nego_words = eval(nego_file.read())
nego_file.close()

In [17]:
print(text)













































sahih al bukhari eng


sahih bukhari volume 1 > book 1: revelation 

book 1: revelation 

volume 1, book 1, number 1: 

narrated 'umar bin al-khattab: 

i heard allah's apostle saying, "the reward of deeds depends upon the intentions and every person 

will get the reward according to what he has intended. so whoever emigrated for worldly benefits or 

for a woman to marry, his emigration was for what he emigrated for." 

volume 1, book 1, number 2: 

narrated 'aisha: 

(the  mother  of  the  faithful  believers)  al-harith  bin  hisham  asked  allah's  apostle  "o  allah's 

apostle! how is the divine inspiration revealed to you?" allah's apostle replied, "sometimes it is (re-

vealed) like the ringing of a bell, this form of inspiration is the hardest of all and then this state 

passes ' off after i have grasped what is inspired. sometimes the angel comes in the form of a man 

and talks to me and i grasp whatever he says." 'aisha added: v

In [18]:
# function returns the negation handled word if it is present
# else returns the word itself
def negationHandling(word):
  if word in nego_words:
    return nego_words[word]
  else:
    return word

In [20]:
text = negationHandling(text)

In [23]:
# remove "\n" from the text
text = text.strip('\n')

In [25]:
# remove "\t" from the text
text = text.replace('\t',"")

In [34]:
 import nltk
 nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [38]:
def preprocessing(text):
  text = re.sub("[\(\[].*?[\)\]]","",text)
  sentences = nltk.sent_tokenize(text)
  tokens = []
  temp = ""
  for sentence in sentences:
    words = nltk.word_tokenize(sentence)
    # removing punctuation 
    punctuations = '"#$%&\'()*+,-/:;<=>@\\^_`{|}~<.>/<?>/<!>'
    words = map(lambda x:x.translate(str.maketrans('','',punctuations)),words)
    # remove empty string
    words = filter(lambda x: len(x)>0, words)
    tokens = tokens + list(words)
    temp = " ".join(word for word in tokens)
  return temp


In [41]:
text = preprocessing(text)

In [43]:
text_file = open("preprocessed_1.txt","w")
text_file.write(text)
text_file.close()


### Coreference Resolution

In [42]:
# mapping pronouns to the appropiate nouns
!pip install neuralcoref

     |████████████████████████████████| 286 kB 4.3 MB/s 
     |████████████████████████████████| 131 kB 56.5 MB/s 
     |████████████████████████████████| 79 kB 7.2 MB/s 
     |████████████████████████████████| 8.5 MB 38.9 MB/s 
     |████████████████████████████████| 127 kB 51.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import spacy
import neuralcoref
# load spacy
nlp = spacy.load("en")
# add neural coref to spacy's pipe
neuralcoref.add_to_pipe(nlp)

In [6]:
# the neuralcoref resolution did not work with location pronoun
def coref_resolution(text):
  """function that executes coreference resolution for given text """
  doc = nlp(text)
  #fetches tokens with whitespaces from spacy document
  tok_list = list(token.text_with_ws for token in doc)
  for cluster in doc._.coref_clusters:
    #get tokens from representative cluster name
    cluster_main_words = set(cluster.main.text.split(' '))
    for coref in cluster:
      if coref != cluster.main: # if coreference element is not the representative element of the cluster
        if coref.text != cluster.main.text and bool (set(coref.text.split(' ')).intersection(cluster_main_words))==False:
          #if coreference element text and representative element text are not equal and none of the coreference element words are in representative element.this was done to handle nested coreference scenarios.
          tok_list[coref.start] = cluster.main.text + \
                                   doc[coref.end-1].whitespace_
          for i in range(coref.start+1,coref.end):
            tok_list[i]=""
  return "".join(tok_list)



